In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import sys
sys.path.append("..")
from functions.functions_analysis import *

# Import data

In [2]:
sumtable = pd.read_excel("../../data/summary_table_all.xlsx")
sumtable.drop('Unnamed: 0', axis = 1,inplace = True)

scraped_data = pd.read_csv("../../data/data_from_scrapy.csv")

#length
len(sumtable),len(scraped_data)

(5771, 5771)

In [3]:
scraped_data.head(3)

,case_brief,case_name,case_status,class_period_end,class_period_start,company_market,court,date_filed,date_of_last_review,docket,...,fic_links_list,fic_summary_table,headquarters,industry,judge,market_status,plaintiffs,sector,ticker_symbol,url
0,"According to the Complaint, Wirecard is a tech...",Wirecard AG Securities Litigation,ONGOING,6/24/20,8/17/15,OTC-BB,E.D. Pennsylvania,7/7/20,7/8/20,20-CV-03326,...,http://securities.stanford.edu/filings-documen...,"{""No"":{""0"":""1"",""1"":""2""},""Document Title"":{""0"":...",Germany,Software & Programming,Hon. ANITA B. BRODY,Public (Listed),"The Rosen Law Firm, P.A. (Jenkintown)",Technology,WCAGY,http://securities.stanford.edu/filings-case.ht...
1,"According to the Complaint, Finjan Holdings, I...","Finjan Holdings, Inc. Securities Litigation",ONGOING,6/29/20,6/10/20,NASDAQ,N.D. California,6/29/20,6/30/20,20-CV-04289,...,http://securities.stanford.edu/filings-documen...,"{""No"":{""0"":""1"",""1"":""2""},""Document Title"":{""0"":...",United States,Software & Programming,Hon. Edward M. Chen,Public (Listed),"Brodsky & Smith, LLC (California)",Technology,FNJN,http://securities.stanford.edu/filings-case.ht...
2,"According to the Complaint, Chembio Diagnostic...","Chembio Diagnostics, Inc. Securities Litigation",ONGOING,7/1/20,6/16/20,NASDAQ,E.D. New York,7/1/20,7/3/20,20-CV-02918,...,http://securities.stanford.edu/filings-documen...,"{""No"":{""0"":""1"",""1"":""2""},""Document Title"":{""0"":...",United States,Biotechnology & Drugs,Hon. Sandra J. Feuerstein,Public (Listed),Fields Kupka & Shukurov LLP\n\t\t\t\t\t\t\t\t\...,Healthcare,CEMI,http://securities.stanford.edu/filings-case.ht...


# Data Prepossing

In [133]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize

In [5]:
sumtable["case_id"] = sumtable.index
scraped_data["case_id"] = scraped_data.index

In [6]:
scraped_data["case_id"] = scraped_data.index
scraped_data.head(2)

,case_brief,case_name,case_status,class_period_end,class_period_start,company_market,court,date_filed,date_of_last_review,docket,...,fic_summary_table,headquarters,industry,judge,market_status,plaintiffs,sector,ticker_symbol,url,case_id
0,"According to the Complaint, Wirecard is a tech...",Wirecard AG Securities Litigation,ONGOING,6/24/20,8/17/15,OTC-BB,E.D. Pennsylvania,7/7/20,7/8/20,20-CV-03326,...,"{""No"":{""0"":""1"",""1"":""2""},""Document Title"":{""0"":...",Germany,Software & Programming,Hon. ANITA B. BRODY,Public (Listed),"The Rosen Law Firm, P.A. (Jenkintown)",Technology,WCAGY,http://securities.stanford.edu/filings-case.ht...,0
1,"According to the Complaint, Finjan Holdings, I...","Finjan Holdings, Inc. Securities Litigation",ONGOING,6/29/20,6/10/20,NASDAQ,N.D. California,6/29/20,6/30/20,20-CV-04289,...,"{""No"":{""0"":""1"",""1"":""2""},""Document Title"":{""0"":...",United States,Software & Programming,Hon. Edward M. Chen,Public (Listed),"Brodsky & Smith, LLC (California)",Technology,FNJN,http://securities.stanford.edu/filings-case.ht...,1


In [7]:
#add Concrete Exchange Information (company_market only listed primary)
def findExchange(ticker):
    return sumtable["Exchange"][sumtable["Ticker"]==ticker].unique()
scraped_data["exchange"]=scraped_data["ticker_symbol"].apply(lambda x: findExchange(x))

In [110]:
#calculate duration
scraped_data["class_period_start"] = pd.to_datetime(scraped_data["class_period_start"])
scraped_data["class_period_end"] = pd.to_datetime(scraped_data["class_period_end"])
scraped_data["date_filed"] = pd.to_datetime(scraped_data["date_filed"])
scraped_data["class_duration"] = scraped_data["class_period_end"] - scraped_data["class_period_start"]

In [185]:
#subset the data to features needed
data = scraped_data[["case_status",
                    "class_period_end",
                    "class_period_start",
                    "company_market",
                    "court",
                    "date_filed",
                    "headquarters",
                    "industry",
                    "judge",
                    "market_status",
                    "sector",
                    "class_duration"]]
data = data.dropna()

In [186]:
#remove ongoing cases and remove judge as a feature
data_input = data[data["case_status"]!="ONGOING"]
data_input = data_input.drop(columns = ["judge"])

In [ ]:
###To do: word to vec similarity of the case brief

In [187]:
#construct X

##creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

##identify columns
category_columns = ["company_market","court","headquarters","industry","market_status"]#,"sector"]

##fit encoder and transfrom
enc.fit(data_input[category_columns].values)
X_onehot = enc.transform(data_input[category_columns].values).toarray() #enc.get_feature_names(category_columns)

In [188]:
##transform time to utc
from datetime import timezone 
import datetime

data_input["class_period_end_num"] = data_input["class_period_end"].apply(lambda x:x.replace(tzinfo = timezone.utc).timestamp())
data_input["class_period_start_num"] = data_input["class_period_start"].apply(lambda x:x.replace(tzinfo = timezone.utc).timestamp())
data_input["date_filed_num"] = data_input["date_filed"].apply(lambda x:x.replace(tzinfo = timezone.utc).timestamp())
data_input["class_duration_num"] = data_input["class_period_end_num"]-data_input["class_period_start_num"]

In [189]:
##get other vars for X
X_time = data_input[["class_period_end_num","class_period_start_num","date_filed_num","class_duration_num"]].values

def time_norm(x_time):
    normalized = np.zeros(shape=(X_time.shape[0],1))
    for i in range(x_time.shape[1]):
        f = np.take(x_time,[i],1)
        f = (f- np.min(f))/np.std(f)
        #print(normalized.shape,f.shape)
        normalized = np.concatenate((normalized,f),1)
    
    normalized = np.delete(normalized,[0],1)
    return normalized
        
X = np.concatenate((time_norm(X_time),X_onehot),1)

In [191]:
#contruct y: label encoder
le = LabelEncoder()
le.fit(data_input['case_status'].unique())
y=le.transform(data_input['case_status'].values).reshape(-1,1)

In [192]:
X.shape,y.shape

((5061, 266), (5061, 1))

In [193]:
#create cross validation sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=777)
X_ttrain, X_tval, y_ttrain, y_tval = train_test_split(X_train, y_train, test_size=0.20, random_state=888)

In [194]:
len(X_ttrain),len(X_tval),len(X_test)

(2833, 709, 1519)

# Decistion Tree

In [195]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [196]:
depth_train = []
depth_val = []

for i in range(3,20):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    
    # Perform 10-fold cross validation 
    clf = clf.fit(X_train,y_train)
    scores_train = clf.score(X_train,y_train)
    scores_val = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, n_jobs=4)
    depth_train.append((i,scores_val.mean()))
    depth_val.append((i,scores_val.mean()))
print(depth_train)
print(depth_val)

[(3, 0.6476557650990691), (4, 0.6530261796769316), (5, 0.6439906103286385), (6, 0.6403214768839023), (7, 0.6420004774409166), (8, 0.6448245404631177), (9, 0.635239118325774), (10, 0.6377775125328241), (11, 0.6386297445691096), (12, 0.6422972865441234), (13, 0.6358009071377417), (14, 0.640325455558208), (15, 0.6406055542293309), (16, 0.6335386329275084), (17, 0.6380599984085303), (18, 0.6394819766053951), (19, 0.6422956950744012)]
[(3, 0.6476557650990691), (4, 0.6530261796769316), (5, 0.6439906103286385), (6, 0.6403214768839023), (7, 0.6420004774409166), (8, 0.6448245404631177), (9, 0.635239118325774), (10, 0.6377775125328241), (11, 0.6386297445691096), (12, 0.6422972865441234), (13, 0.6358009071377417), (14, 0.640325455558208), (15, 0.6406055542293309), (16, 0.6335386329275084), (17, 0.6380599984085303), (18, 0.6394819766053951), (19, 0.6422956950744012)]


In [197]:
# plot duration with 
# stepwise logistic regression

## Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification